In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import torch
import os
import numpy as np
import requests
from io import BytesIO
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler

if torch.cuda.is_available():
    print("GPU detected: " + torch.cuda.get_device_name(0))
    DEVICE = "cuda"
else:
    print("No GPU detected. Running on CPU (will be slow).")
    DEVICE = "cpu"

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


GPU detected: NVIDIA A100-SXM4-40GB


In [ ]:
# --- CONFIGURATION ---

# Path to your trained checkpoint
CONTROLNET_PATH = "akash00chaudhari/Anime-70"

# Base Model
BASE_MODEL = "stablediffusionapi/anything-v3"

# Google Drive Links for Examples
EXAMPLE_LIST = [
    ["https://drive.google.com/uc?export=view&id=1S8dGygbKVuUJfBSAxSRly-xjhEdKc8ej", "a man with a big smile on his face"],
    ["https://drive.google.com/uc?export=view&id=1G7387x4BlHuxaYQhF3i_dDTjVEpt6lqE", "a man with a white hat and a red cross on his head"],
]

In [ ]:
print(f"Loading models")

# 1. Load ControlNet
# We do NOT use torch_dtype=torch.float16 here to prevent type conflicts
controlnet = ControlNetModel.from_pretrained(
    CONTROLNET_PATH
).to(DEVICE)

# 2. Load Pipeline
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL,
    controlnet=controlnet,
    safety_checker=None
).to(DEVICE)

# 3. Optimize Scheduler
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# 4. Optimize Memory
try:
    pipe.enable_xformers_memory_efficient_attention()
    print("xformers enabled.")
except:
    print("xformers not found (running standard attention).")

print("Models loaded successfully!")

Loading models


config.json: 0.00B [00:00, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

xformers not found (running standard attention).
Models loaded successfully!


In [ ]:
# --- GENERATION FUNCTION ---
def generate_image(input_sketch, prompt, negative_prompt, num_steps, guidance_scale, control_scale):

    # 1. Validation
    if input_sketch is None:
        raise gr.Error("Please upload a sketch or select an example!")

    # 2. Image Loading
    try:
        if isinstance(input_sketch, str):
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(input_sketch, headers=headers)
            input_sketch = Image.open(BytesIO(response.content)).convert("RGB")
        elif isinstance(input_sketch, np.ndarray):
            input_sketch = Image.fromarray(input_sketch).convert("RGB")
    except Exception as e:
        print(f"Error loading image: {e}")
        raise gr.Error("Could not load input image.")

    # 3. Preprocessing
    input_sketch = input_sketch.resize((512, 512))

    # 4. Prompt Handling
    final_prompt = prompt
    if not final_prompt or final_prompt.strip() == "":
        final_prompt = "best quality, high quality, masterpiece, anime style"

    # 5. Inference
    generator = torch.Generator(device=DEVICE).manual_seed(42)

    try:
        output = pipe(
            prompt=final_prompt,
            image=input_sketch,
            negative_prompt=negative_prompt,
            num_inference_steps=int(num_steps),
            guidance_scale=float(guidance_scale),
            controlnet_conditioning_scale=float(control_scale),
            generator=generator
        ).images[0]

        return output
    except Exception as e:
        print(f"Generation Failed: {e}")
        raise gr.Error(f"Error: {e}")

# --- GRADIO UI ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎨 Sketch-to-Anime Generator")
    gr.Markdown("Upload a sketch or click an example below to generate.")

    with gr.Row():
        with gr.Column():
            # Inputs
            input_image = gr.Image(label="Upload Sketch", type="numpy", height=400)
            prompt_input = gr.Textbox(label="Prompt", placeholder="e.g., 1girl, blue hair...")

            with gr.Accordion("Advanced Options", open=False):
                neg_prompt = gr.Textbox(label="Negative Prompt", value="low quality, bad anatomy, worst quality, monochrome")
                steps_slider = gr.Slider(10, 100, value=30, label="Steps")
                cfg_slider = gr.Slider(1, 20, value=7.5, label="Guidance Scale")
                control_slider = gr.Slider(0.0, 2.0, value=1.0, label="Control Strength")

            generate_btn = gr.Button("Generate", variant="primary")

        with gr.Column():
            # Outputs
            output_image = gr.Image(label="Result", height=400)

    # Examples Section
    if len(EXAMPLE_LIST) > 0:
        gr.Markdown("### 📝 Try an Example")
        gr.Examples(
            examples=EXAMPLE_LIST,
            inputs=[input_image, prompt_input],
            run_on_click=False
        )

    # Wiring
    generate_btn.click(
        fn=generate_image,
        inputs=[input_image, prompt_input, neg_prompt, steps_slider, cfg_slider, control_slider],
        outputs=output_image
    )

/tmp/ipython-input-161498030.py:48: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


In [ ]:
print("Launching Gradio...")
demo.launch(share=True, debug=True)

Launching Gradio...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://269382f103495944d2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/30 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://269382f103495944d2.gradio.live
